# AES

In [67]:
def display(matrix):
    for row in matrix:
        print(" ".join(hex(x)[2:].upper().zfill(2) for x in row))
    print()

s_box = [
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5, 0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0, 0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc, 0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a, 0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0, 0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b, 0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85, 0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5, 0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17, 0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88, 0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c, 0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9, 0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6, 0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e, 0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94, 0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68, 0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
]



### SHIFTING ROWS

In [68]:
def substitute(state):
    new_state = []
    for row in state:
        new_row = [s_box[b] for b in row]
        new_state.append(new_row)
    return new_state


In [69]:
def shift_rows(state):
    state[1] = state[1][1:] + state[1][:1]
    state[2] = state[2][2:] + state[2][:2]
    state[3] = state[3][3:] + state[3][:3]
    return state


In [70]:
def gf_mult(a, b):
    result = 0
    for _ in range(8):
        if b & 1:
            result ^= a
        hi = a & 0x80
        a = (a << 1) & 0xFF
        if hi:
            a ^= 0x1b
        b >>= 1
    return result


In [71]:
def mix_columns(state):
    M = [
        [2, 3, 1, 1],
        [1, 2, 3, 1],
        [1, 1, 2, 3],
        [3, 1, 1, 2]
    ]
    new_state = [[0]*4 for _ in range(4)]
    for i in range(4):
        for j in range(4):
            new_state[i][j] = (
                gf_mult(state[0][j], M[i][0]) ^
                gf_mult(state[1][j], M[i][1]) ^
                gf_mult(state[2][j], M[i][2]) ^
                gf_mult(state[3][j], M[i][3])
            )
    return new_state


In [72]:
def add_round_key(state):
    key = [
        [0xac, 0x19, 0x28, 0x57],
        [0x77, 0xfa, 0xd1, 0x5c],
        [0x66, 0xdc, 0x29, 0x00],
        [0xf3, 0x21, 0x41, 0x6a]
    ]
    for i in range(4):
        for j in range(4):
            state[i][j] ^= key[i][j]
    return state

data = [
    [0xea, 0x04, 0x65, 0x85],
    [0x83, 0x45, 0x5d, 0x96],
    [0x5c, 0x33, 0x98, 0xb0],
    [0xf0, 0x2d, 0xad, 0xc5]
]


In [73]:
print("Initial:")
display(data)

data = substitute(data)
print("After Substitution:")
display(data)

data = shift_rows(data)
print("After Shift Rows:")
display(data)

data = mix_columns(data)
print("After Mix Columns:")
display(data)

data = add_round_key(data)
print("After Add Round Key:")
display(data)

for _ in range(8):
    data = add_round_key(mix_columns(shift_rows(substitute(data))))

data = add_round_key(shift_rows(substitute(data)))
print("Final Output:")
display(data)

Initial:
EA 04 65 85
83 45 5D 96
5C 33 98 B0
F0 2D AD C5

After Substitution:
87 F2 4D 97
EC 6E 4C 90
4A C3 46 E7
8C D8 95 A6

After Shift Rows:
87 F2 4D 97
6E 4C 90 EC
46 E7 4A C3
A6 8C D8 95

After Mix Columns:
47 40 A3 4C
37 D4 70 9F
94 E4 3A 42
ED A5 A6 BC

After Add Round Key:
EB 59 8B 1B
40 2E A1 C3
F2 38 13 42
1E 84 E7 D6

Final Output:
EC 8D 53 B3
C4 89 08 C4
57 23 9B AE
82 6E C2 A1

